In [ ]:
#!pip install GitPython
#!pip install sagemaker
#!pip install s3fs
#!pip install aws-mlops
import os
import sys
#sys.path.insert(1, os.getcwd() + '/..')
import pandas as pd
import config as config
from aws_mlops.data_storage import DataStorage

In [ ]:
# object for data storage management
studio = DataStorage(config.source_bucket, config.test_data_key)
config_name = 'testing_input'

In [ ]:
# load dataframes
[columns_names, target, identifier] = studio.restore_test([config.target, config.identifier])
test = studio.restore(config.test_filename, config.test_path)
score = studio.restore(config.prediction_filename, config.prediction_path)

In [ ]:
# join datasets
df = pd.concat([identifier, target, score, test], axis=1)
df.columns = [config.identifier, config.target, config.score] + list(columns_names[0])

In [ ]:
# prediction datasets
prediction = df[[config.identifier, config.score]]

In [ ]:
# report datasets
report = pd.DataFrame({
    'Mean squared error': [metrics.mean_squared_error(df[config.target], df[config.score])],
    'Mean absolute error': [metrics.mean_absolute_error(df[config.target], df[config.score])]
})

In [ ]:
# output - dataframes: prediction and report
output_dataframes = [prediction, report]
for output in getattr(config, config_name)['outputs']:
    dataframe = output_dataframes.pop(0)
    ds.local_save(dataframe, output['S3Output']['LocalPath'], output['OutputName'], header = True, index = False)